# VAE
Here we implement a VAE using Lightning.


## Import packages

In [2]:
! pip list | grep pytorch

In [8]:
%%capture
! pip install pytorch-lightning==1.4.4
! pip install pytorch-lightning-bolts==0.2.5

In [9]:
import pytorch_lightning as pl
import torch
from torch import nn
from torch.nn import functional as F
from pl_bolts.models.autoencoders.components import resnet18_decoder,resnet18_encoder

## Model

In [10]:
class VAE(pl.LightningModule):
    def __init__(self, enc_out_dim=512, latent_dim=256, input_height=32):
        super().__init__()

        self.save_hyperparameters()

        # encoder, decoder
        self.encoder = resnet18_encoder(False, False)
        self.decoder = resnet18_decoder(
            latent_dim=latent_dim, 
            input_height=input_height, 
            first_conv=False, 
            maxpool1=False
        )

        # distribution parameters
        self.fc_mu = nn.Linear(enc_out_dim, latent_dim)
        self.fc_var = nn.Linear(enc_out_dim, latent_dim)

        # for the gaussian likelihood
        self.log_scale = nn.Parameter(torch.Tensor([0.0]))

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4)

    def gaussian_likelihood(self, mean, logscale, sample):
        scale = torch.exp(logscale)
        dist = torch.distributions.Normal(mean, scale)
        log_pxz = dist.log_prob(sample)
        return log_pxz.sum(dim=(1, 2, 3))

    def kl_divergence(self, z, mu, std):
        # --------------------------
        # Monte carlo KL divergence
        # --------------------------
        # 1. define the first two probabilities (in this case Normal for both)
        p = torch.distributions.Normal(torch.zeros_like(mu), torch.ones_like(std))
        q = torch.distributions.Normal(mu, std)

        # 2. get the probabilities from the equation
        log_qzx = q.log_prob(z)
        log_pz = p.log_prob(z)

        # kl
        kl = (log_qzx - log_pz)
        kl = kl.sum(-1)
        return kl

    def training_step(self, batch, batch_idx):
        x, _ = batch

        # encode x to get the mu and variance parameters
        x_encoded = self.encoder(x)
        self.mu, log_var = self.fc_mu(x_encoded), self.fc_var(x_encoded)

        # sample z from q
        self.std = torch.exp(log_var / 2)
        q = torch.distributions.Normal(self.mu, self.std)
        z = q.rsample()

        # decoded
        x_hat = vae.decoder(z)

        # reconstruction loss
        recon_loss = self.gaussian_likelihood(x_hat, self.log_scale, x)

        # kl
        kl = self.kl_divergence(z, self.mu, self.std)

        # elbo
        elbo = (kl - recon_loss)
        elbo = elbo.mean()

        #self.log("my_loss", recon_loss.mean(), on_step=True, on_epoch=True, prog_bar=True, logger=True)

        self.log_dict({
            'elbo': elbo,
            'kl': kl.mean(),
            'recon_loss': recon_loss.mean(), 
            'reconstruction': recon_loss.mean(),
            'kl': kl.mean(),
        })

        return elbo

## Data

Let's use CIFAR-10 already split up and transformed.

The Lightning Datamodule has 3 dataloaders, train, val, test

In [ ]:
#%%capture
#! pip install pytorch-lightning-bolts==0.2.5

In [11]:
from pl_bolts.datamodules.cifar10_datamodule import CIFAR10DataModule
data = CIFAR10DataModule('.', batch_size=32)

## Training

In [12]:
%%time
from pytorch_lightning.loggers import TensorBoardLogger

pl.seed_everything(1234)

vae = VAE()
logger = TensorBoardLogger("lightning_logs/")
trainer = pl.Trainer(gpus=1, max_epochs=1, progress_bar_refresh_rate=10, logger=logger)
history = trainer.fit(vae, data)

## Image generation

In [13]:
from matplotlib.pyplot import imshow, figure
import numpy as np
from torchvision.utils import make_grid
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
figure(figsize=(8, 3), dpi=300)

num_preds = 32

# Z COMES FROM NORMAL(0, 1)
mu = torch.zeros((1,256))
std = torch.ones((1,256))
p = torch.distributions.Normal(mu, std)
z = p.rsample((num_preds,))

# SAMPLE IMAGES
with torch.no_grad():
    pred = vae.decoder(z.to(vae.device)).cpu()

# UNDO DATA NORMALIZATION
normalize = cifar10_normalization()
mean, std = np.array(normalize.mean), np.array(normalize.std)
img = make_grid(pred).permute(1, 2, 0).numpy() * std + mean

# PLOT IMAGES
imshow(img);